## Sprint11 深層学習スクラッチ 畳み込みニューラルネットワーク1
#### 1.このSprintについて
##### Sprintの目的
スクラッチを通してCNNの基礎を理解する

##### どのように学ぶか
スクラッチで1次元用畳み込みニューラルネットワークを実装した後、学習と検証を行なっていきます。

#### 2.1次元の畳み込みニューラルネットワークスクラッチ
畳み込みニューラルネットワーク（CNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

このSprintでは1次元の 畳み込み層 を作成し、畳み込みの基礎を理解することを目指します。次のSprintでは2次元畳み込み層とプーリング層を作成することで、一般的に画像に対して利用されるCNNを完成させます。

クラスの名前はScratch1dCNNClassifierとしてください。クラスの構造などは前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

##### 1次元畳み込み層とは
CNNでは画像に対しての2次元畳み込み層が定番ですが、ここでは理解しやすくするためにまずは1次元畳み込み層を実装します。1次元畳み込みは実用上は自然言語や波形データなどの 系列データ で使われることが多いです。

畳み込みは任意の次元に対して考えることができ、立体データに対しての3次元畳み込みまではフレームワークで一般的に用意されています。

##### データセットの用意
検証には引き続きMNISTデータセットを使用します。1次元畳み込みでは全結合のニューラルネットワークと同様に平滑化されたものを入力します。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix

### 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成
チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成してください。基本構造は前のSprintで作成した全結合層のFCクラスと同じになります。なお、重みの初期化に関するクラスは必要に応じて作り変えてください。Xavierの初期値などを使う点は全結合層と同様です。

ここでは パディング は考えず、ストライド も1に固定します。また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応してください。この部分の拡張はアドバンス課題とします。

フォワードプロパゲーションの数式は以下のようになります。  
$$\alpha_i = \sum_{s=0}^{F-1}x_{(i+s)}W_s + b$$

$a_i$ : 出力される配列のi番目の値

$F$ : フィルタのサイズ

$x_{(i+s})$ : 入力の配列の(i+s)番目の値

$w_s$ : 重みの配列のs番目の値

$b$ : バイアス項

全てスカラーです。

次に更新式です。ここがAdaGradなどに置き換えられる点は全結合層と同様です。  
$$w'_s = w_s -\alpha\frac{\partial L}{\partial w_s}$$$$b' = b - \alpha\frac{\partial L}{\partial b}$$

$\alpha$ : 学習率

$\frac{\partial L}{\partial w_s}$ : w_s に関する損失 $L$ の勾配

$\frac{\partial L}{\partial b}$ : $b$ に関する損失 $L$ の勾配

勾配 $\frac{\partial L}{\partial w_s}$ や $\frac{\partial L}{\partial b}$ を求めるためのバックプロパゲーションの数式が以下です。

$$\frac{\partial L}{\partial w_s} = \sum_{i=0}^{N_{out}-1}\frac{\partial L}{\partial a_i}x_{(i+s}$$$$\frac{\partial L}{\partial b} =  \sum_{i=0}^{N_{out}-1}\frac{\partial L}{\partial a_i}$$

$\frac{\partial L}{\partial a_i}$: 勾配の配列のi番目の値

$N_{out} : 出力のサイズ

前の層に流す誤差の数式は以下です。　　
$$\frac{\partial L}{\partial x_j} = \sum_{s=0}^{F-1}\frac{\partial L}{\partial a_{j-s}}w_s$$

$\frac{\partial L}{\partial x_j}$ : 前の層に流す誤差の配列のj番目の値

ただし、 $j−s&lt;0$ または$j−s&gt;N_{out}−1$ のとき$\frac{\partial L}{\partial a_{j-s}}=0$ です。

全結合層との大きな違いは、重みが複数の特徴量に対して共有されていることです。この場合は共有されている分の誤差を全て足すことで勾配を求めます。計算グラフ上での分岐はバックプロパゲーションの際に誤差の足し算をすれば良いことになります。

### 【問題2】1次元畳み込み後の出力サイズの計算¶
畳み込みを行うと特徴量の数が変化します。どのように変化するかは以下の数式から求められます。パディングやストライドも含めています。この計算を行う関数を作成してください。  
$$N_{out} = \frac{N_{in} + 2P - F}{S} + 1$$

$N_{out}$ : 出力のサイズ（特徴量の数）

$N_{in}$ : 入力のサイズ（特徴量の数）

$P$ : ある方向へのパディングの数

$F$ : フィルタのサイズ

$S$ : ストライドのサイズ

In [ ]:
# サンプルデータ
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])
delta_a = np.array([10, 20])

In [2]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.optimizer = optimizer
        #self.W = initializer.W(self.n_nodes1, self.n_nodes2)
        #self.B = initializer.B(self.n_nodes2)
        self.W = np.array([3,5,7]) 
        self.B = np.array([1])
        self.P = 0
        self.Str = 1
        self.s = len(self.W)
        self.a = np.array([])
        self.dW = np.array([])
        self.dX = np.array([])

    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes_bf)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes_af)
            出力
        """
        #self.X = X
        self.X = X
        self.Xsize = len(self.X)
        self._output_size()
        
        self.a = np.append(self.a, np.array([(self.X[i:i+self.s] @ self.W.T + self.B) for i in range(self.Nout)]))
        return self.a
    
    def _output_size(self):
        self.Nout = int((len(self.X) + 2*self.P - self.s) / self.Str + 1)       
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        self.dB = np.sum(dA, axis=0)
        self.dW = np.append(self.dW, np.array([(dA @ self.X[i:i+self.Nout].T) for i in range(self.s)]))

        for j in range(len(self.X)):
            if j-self.Nout < 0:
                #display(j)
                #display(np.flip(self.W[ : self.s-(self.Nout-j) ]))
                #display(dA[:j+1])
                self.dX = np.append(self.dX, (dA[:j+1] @ np.flip(self.W[ : self.s-(self.Nout-j) ]).T))
                
            elif j > len(self.X) - self.Nout:
                #display(j)
                #display(np.flip(dA    [-(j-(self.Xsize - self.Nout)):]))
                #display(np.flip(self.W[-(j-(self.Xsize - self.Nout)):]))
                self.dX = np.append(self.dX, (np.flip(dA    [-(j-(self.Xsize - self.Nout)):])) @ 
                                              np.flip(self.W[-(j-(self.Xsize - self.Nout)):]).T)
            else:
                #display(j)
                #display(np.flip(self.W[ j-self.Nout+1 : j+1 ]))
                self.dX = np.append(self.dX, (dA @ np.flip(self.W[ j-self.Nout+1 : j+1 ]).T))

        # 更新
        #self.W = self.optimizer.update(dW, self.W)
        #self.B = self.optimizer.update(dB, self.B)

        return self.dX


### 【問題3】小さな配列での1次元畳み込み層の実験
次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認してください。

入力$x$、重み$w$、バイアス$b$を次のようにします。

x = np.array([1,2,3,4])  
w = np.array([3, 5, 7])  
b = np.array([1])  
フォワードプロパゲーションをすると出力は次のようになります。

a = np.array([35, 50])  
次にバックプロパゲーションを考えます。誤差は次のようであったとします。

delta_a = np.array([10, 20])  
バックプロパゲーションをすると次のような値になります。

delta_b = np.array([30])  
delta_w = np.array([50, 80, 110])  
delta_x = np.array([30, 110, 170, 140])  
##### 実装上の工夫
畳み込みを実装する場合は、まずはfor文を重ねていく形で構いません。しかし、できるだけ計算は効率化させたいため、以下の式を一度に計算する方法を考えることにします。  
$$a_i = \sum_{s=0}^{F-1}x_{i+s}w_s + b$$

バイアス項は単純な足し算のため、重みの部分を見ます。$$\sum_{s=0}^{F-1}x_{i+s}w_s$$

これは、xの一部を取り出した配列とwの配列の内積です。具体的な状況を考えると、以下のようなコードで計算できます。この例では流れを分かりやすくするために、各要素同士でアダマール積を計算してから合計を計算しています。これは結果的に内積と同様です。

x = np.array([1, 2, 3, 4])  
w = np.array([3, 5, 7])

a = np.empty((2, 3))  

indexes0 = np.array([0, 1, 2]).astype(np.int)  
indexes1 = np.array([1, 2, 3]).astype(np.int)

a[0] = x[indexes0]*w # x[indexes0]は([1, 2, 3])である
a[1] = x[indexes1]*w # x[indexes1]は([2, 3, 4])である  

a = a.sum(axis=1)  
ndarrayは配列を使ったインデックス指定ができることを利用した方法です。

また、二次元配列を使えば一次元配列から二次元配列が取り出せます。

x = np.array([1, 2, 3, 4])  
indexes = np.array([[0, 1, 2], [1, 2, 3]]).astype(np.int)

print(x[indexes]) # ([[1, 2, 3], [2, 3, 4]])  
このこととブロードキャストなどをうまく組み合わせることで、一度にまとめて計算することも可能です。

畳み込みの計算方法に正解はないので、自分なりに効率化していってください。

##### 《参考》
以下のページのInteger array indexingの部分がこの方法についての記述です。

Indexing — NumPy v1.17 Manual

In [3]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])
test = FC(1,2,3,4)

In [4]:
test.forward(x)

array([35., 50.])

In [5]:
delta_a = np.array([10, 20])
test.backward(delta_a)

array([ 30., 110., 170., 140.])

In [6]:
display(test.dB)
display(test.dW)

30

array([ 50.,  80., 110.])

In [7]:
class Conv1d:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.optimizer = optimizer
        #self.W = initializer.W(self.n_nodes1, self.n_nodes2)
        #self.B = initializer.B(self.n_nodes2)
        self.W = np.array([3,5,7]) 
        self.B = np.array([1])
        self.P = 0
        self.Str = 1
        self.s = len(self.W)
        self.a = np.array([])
        self.dW = np.array([])
        self.dX = np.array([])

    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes_bf)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes_af)
            出力
        """
        #self.X = X
        self.X = X
        self.Xsize = len(self.X)
        self._output_size()
        
        self.a = np.append(self.a, np.array([(self.X[i:i+self.s] @ self.W.T + self.B) for i in range(self.Nout)]))
        return self.a
    
    def _output_size(self):
        self.Nout = int((len(self.X) + 2*self.P - self.s) / self.Str + 1)       
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        self.dB = np.sum(dA, axis=0)
        self.dW = np.append(self.dW, np.array([(dA @ self.X[i:i+self.Nout].T) for i in range(self.s)]))

        for j in range(len(self.X)):
            if j-self.Nout < 0:
                #display(j)
                #display(np.flip(self.W[ : self.s-(self.Nout-j) ]))
                #display(dA[:j+1])
                self.dX = np.append(self.dX, (dA[:j+1] @ np.flip(self.W[ : self.s-(self.Nout-j) ]).T))
                
            elif j > len(self.X) - self.Nout:
                #display(j)
                #display(np.flip(dA    [-(j-(self.Xsize - self.Nout)):]))
                #display(np.flip(self.W[-(j-(self.Xsize - self.Nout)):]))
                self.dX = np.append(self.dX, (np.flip(dA    [-(j-(self.Xsize - self.Nout)):])) @ 
                                              np.flip(self.W[-(j-(self.Xsize - self.Nout)):]).T)
            else:
                #display(j)
                #display(np.flip(self.W[ j-self.Nout+1 : j+1 ]))
                self.dX = np.append(self.dX, (dA @ np.flip(self.W[ j-self.Nout+1 : j+1 ]).T))

        # 更新
        #self.W = self.optimizer.update(dW, self.W)
        #self.B = self.optimizer.update(dB, self.B)

        return self.dX


### 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成 
チャンネル数を1に限定しない1次元畳み込み層のクラスConv1dを作成してください。

例えば以下のようなx, w, bがあった場合は、

x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。  
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。  
b = np.array([1, 2, 3]) # （出力チャンネル数）  
出力は次のようになります。

a = np.array([[16, 22], [17, 23], [18, 24]]) # shape(3, 2)で、（出力チャンネル数、特徴量数）である。  
入力が2チャンネル、出力が3チャンネルの例です。計算グラフを書いた上で、バックプロパゲーションも手計算で考えてみましょう。計算グラフの中には和と積しか登場しないので、微分を新たに考える必要はありません。

##### 《補足》
チャンネル数を加える場合、配列をどういう順番にするかという問題があります。(バッチサイズ、チャンネル数、特徴量数)または(バッチサイズ、特徴量数、チャンネル数)が一般的で、ライブラリによって順番は異なっています。（切り替えて使用できるものもあります）

今回のスクラッチでは自身の実装上どちらが効率的かを考えて選んでください。上記の例ではバッチサイズは考えておらず、(チャンネル数、特徴量数)です。

In [8]:
import itertools
import pprint

In [9]:
class Conv1d:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self):
        self.P = 0
        self.Str = 1
        self.a = np.array([])
        self.dW = np.array([])
        self.dX = np.array([])
        #self.s=None
        
    def forward(self, X, W ,B):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes_bf)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes_af)
            出力
        """
        self.X = X
        self.Xsize = self.X.shape[1]
        self.W = W 
        self.B = B
        self.s = self.W.shape[2]
        
        self._output_size()
        self.a = np.append(self.a, np.array([np.sum(self.X[:, i:i+self.s] * self.W[i0,:,:]) + self.B[i0] 
                                             for i0,i in itertools.product(range(self.W.shape[0]), range(self.Nout))]))
        self.a = self.a.reshape(self.W.shape[0], self.Nout)
        return self.a
    
    def _output_size(self):
        self.Nout = int((self.Xsize + 2*self.P - self.s) / self.Str + 1)       
    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        self.dB = np.sum(dA, axis=1)   
        
        for i0 in range(self.W.shape[0]):
            dX_ary = np.zeros((self.W.shape[1], self.W.shape[2]))
            for i in range(self.Nout):
                dX_ary += dA[i0,i] * self.X[:, i:i+self.s]
            self.dW = np.append(self.dW, dX_ary)
        self.dW = self.dW.reshape(3,2,3)
        """検討中
        self.Xp=np.array([self.X]*self.W.shape[0])
        self.
        self.dW= np.zeros((self.W.shape))
        self.dW = [self.dW + (dA * self.Xp[:,:,i:i+self.s]) for i in range(self.Nout)]
        """
        #self.dW = np.append(self.dW, [dX_ary + (np.ones((self.W[1].shape, self.W[2].shape))*dA[i0,i]) * self.X[:, i:i+self.Nout] 
        #                              for i in range(self.Nout)] for i0 in range(self.W.shape[0]))
        
        #for j in range(len(self.X)):
        #    if j-self.Nout < 0:
        #        self.dX = np.append(self.dX, (dA[:j+1] @ np.flip(self.W[ : self.s-(self.Nout-j) ]).T))
        #        
        #    elif j > len(self.X) - self.Nout:
        #        self.dX = np.append(self.dX, (np.flip(dA    [-(j-(self.Xsize - self.Nout)):])) @ 
        #                                      np.flip(self.W[-(j-(self.Xsize - self.Nout)):]).T)
        #    else:
        #        self.dX = np.append(self.dX, (dA @ np.flip(self.W[ j-self.Nout+1 : j+1 ]).T))

        return self.dW
    
    #def _naiseki_plus(self,dA):
    #    dX_ary = np.zeros((self.W[1].shape, self.W[2].shape))
    #    dX_ary = [dX_ary + (np.ones((self.W[1].shape, self.W[2].shape))*dA[i0,i]) * self.X[:, i:i+self.Nout] for i in range(self.Nout)]
    #    return dX_ary

In [10]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）

In [11]:
test2dim = Conv1d()
test2dim.forward(x,w,b)

array([[16., 22.],
       [17., 23.],
       [18., 24.]])

In [12]:
x = np.array([[2, 3, 4, 5], [1, 2, 3, 4]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
#w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
w = np.array([[[1,1,1],
            [1,1,1]],
            [[1,1,1],
            [2,1,1]],
            [[2,1,1],
            [1,1,2]]])
b = np.array([3, 2, 1]) # （出力チャンネル数）

In [13]:
test2dim = Conv1d()
test2dim.forward(x,w,b)

array([[18., 24.],
       [18., 25.],
       [21., 29.]])

### バックプロパゲーションの確認

In [14]:
dA = np.array([[52,56],
            [32,35],
            [9,11]])
#dA = np.array([9,11])

In [15]:
dA3=np.array([dA]*3)
dA3

array([[[52, 56],
        [32, 35],
        [ 9, 11]],

       [[52, 56],
        [32, 35],
        [ 9, 11]],

       [[52, 56],
        [32, 35],
        [ 9, 11]]])

In [16]:
dA3=dA3.transpose(1,2,0)
dA3

array([[[52, 52, 52],
        [56, 56, 56]],

       [[32, 32, 32],
        [35, 35, 35]],

       [[ 9,  9,  9],
        [11, 11, 11]]])

In [17]:
test2dim.backward(dA)

array([[[272., 380., 488.],
        [164., 272., 380.]],

       [[169., 236., 303.],
        [102., 169., 236.]],

       [[ 51.,  71.,  91.],
        [ 31.,  51.,  71.]]])

### 【問題8】学習と推定 
これまで使ってきたニューラルネットワークの全結合層の一部をConv1dに置き換えてMNISTを学習・推定し、Accuracyを計算してください。
出力層だけは全結合層をそのまま使ってください。ただし、チャンネルが複数ある状態では全結合層への入力は行えません。その段階でのチャンネルは1になるようにするか、 平滑化 を行なってください。
画像に対しての1次元畳み込みは実用上は行わないことのため、精度は問いません。

In [18]:
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
(X_train, t_train), (X_test, t_test) = mnist.load_data()

C:\Users\anai\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
X_train  = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
enc = OneHotEncoder(handle_unknown="ignore", sparse=False)
t_train_one_hot = enc.fit_transform(t_train[:, np.newaxis])
t_test_one_hot = enc.fit_transform(t_test[:,  np.newaxis])

In [20]:
X_train = X_train.reshape(60000, 1, 784)
X_test = X_test.reshape(-1, 1, 784)
w = np.ones((3, 1, 3))
b = np.array([1, 2, 3])

In [21]:
class CNN_mnist_FC:
    """
    畳み込み層
    Parameters
    ----------
    w:畳み込み層の重み　w.shape  (出力チャネル、入力チャネル、フィルターサイズ)
    b:畳み込み層のバイアス　b.shape (出力チャネル, )
    stride:ストライド数
    padding:パディング数
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, w, b , optimizer,stride, padding):
        self.optimizer = optimizer
        self.W = w
        self.B = b
        self.stride = stride
        self.padding = padding
    
    

    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.A = X
        output_size, chanel_size, filter_size = self.W.shape
        feature_size = self.A.shape[2]
        sample_size = self.A.shape[0]

        a = np.zeros([sample_size, output_size, feature_size-2])
        for samples in range(sample_size):
            for output in range(output_size):
                for j in range(filter_size - 1):
                    sig = 0
                    for chanel in range(chanel_size):
                        for i in range(filter_size):
                            sig += X[samples, chanel, i+j] * self.W[output, chanel, j]
                    a[samples, output, j] = sig + b[output]
        
        return a

    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        self.n_out = N_OUT(self.stride, self.padding, self.W, self.A)
        
        output_size, chanel_size, filter_size = self.W.shape
        feature_size = self.A.shape[2]
        sample_size = self.A.shape[0]
        
        #LBの計算
        self.LB = dA.sum(axis=0)
        self.LB = self.LB.sum(axis=1)
        
        #LWの計算
        self.LW = np.zeros_like(self.W)
        for samples in range(sample_size):
            for output in range(output_size):
                for chanel in range(chanel_size):
                    for i in range(filter_size):
                        for j in range(filter_size -1):
                            self.LW[output, chanel, i] += dA[samples, output, j]*self.A[samples, chanel, j+i]
                        
                        

                    
                    
        #dZの計算
        dZ = np.zeros_like(self.A)
        for samples in range(sample_size):
            for output in range(output_size):
                for chanel in range(chanel_size):
                    for j in range(feature_size):
                        sigma=0
                        for s in range(filter_size):
                            if j - s < 0 or j - s > self.n_out -1:
                                pass
                            else:
                                sigma += dA[samples, output,  j-s] * self.W[output, chanel, s]
                        dZ[samples, chanel, j] += sigma


        
        
        # 更新
        self = self.optimizer.update(self)
        return dZ

In [22]:
class Relu:
    def forward(self, X):
        self.A = X
        return np.maximum(0, X)
    
    def backward(self, Z):
        
        return Z * np.maximum(np.sign(self.A), 0)

In [23]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        init = initializer
        self.n_nodes1 = n_nodes1
        self.W = init.W(n_nodes1, n_nodes2)
        self.B = init.B(n_nodes2)
    

    
    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """
        self.z = X
        self.a = X@self.W + self.B
        
        return self.a

    
    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        dZ = dA @ self.W.T
        self.LW = self.z.T @ dA
        self.LB = np.sum(dA, axis=0)
        
        
        # 更新
        self = self.optimizer.update(self)
        return dZ

In [24]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        
        return W
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B  = self.sigma * np.random.randn(n_nodes2)
        return B

In [25]:
class Softmax:
    def forward(self, A):
        exp_a = np.exp(A)
        softmax_result = np.empty((A.shape[0], A.shape[1]))
        exp_sum = np.sum(exp_a, axis=1)
        for i in range(A.shape[0]):
            softmax_result[i] = exp_a[i] / exp_sum[i]
            
        return softmax_result
    
    def backward(self, Z, Y):
        
        L_A = Z - Y
        self.cross_entropy = -np.average(np.sum(Y*np.log(Z), axis=1))
        
        
        return L_A

In [27]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス

        Returns
        ----------
        layer : 更新後の層のインスタンス
        """

        
        layer.W = layer.W -  self.lr * layer.LW
        
        layer.B = layer.B - self.lr*layer.LB

        
        
        
        return layer

In [29]:
def N_OUT(stride, padding, X,  W):
    if X.ndim == 1:
        return int((X.shape[0] + (2*padding) - len(W) / stride) + 1)
    elif X.ndim == 3:
        return int((X.shape[2] + (2*padding) - len(W) / stride) + 1 )

In [30]:
cnn_mnist = CNN_mnist_FC(w, b, SGD(0.1), 1, 0)
A = cnn_mnist.forward(X_train)
relu = Relu()
A_relu = relu.forward(A)
A_flat = A_relu.reshape(A_relu.shape[0], -1)
FC_1 = FC(2346, 10, SimpleInitializer(0.1), SGD(0.1))
A_FC_1 = FC_1.forward(A_flat)
softmax = Softmax()
A_soft = softmax.forward(A_FC_1)
A_delta = softmax.backward(A_soft, t_train_one_hot)
delta_Z = FC_1.backward(A_delta)
delta_Z_reshape = delta_Z.reshape(A_relu.shape)
delta_Z_relu = relu.backward(delta_Z_reshape)
dZ = cnn_mnist.backward(delta_Z_relu)

In [31]:
X_test = X_test.reshape(-1, 1, 784)
t_A = cnn_mnist.forward(X_test)
t_A = relu.forward(t_A)
t_A  = t_A.reshape(t_A.shape[0], -1)
t_A = FC_1.forward(t_A)
C = np.max(t_A, axis=1)
for i in range(t_A.shape[0]):
    t_A[i] = np.exp(t_A[i] - C[i])
t_A = softmax.forward(t_A)
y = np.argmax(t_A, axis=1)
from sklearn.metrics import accuracy_score
print(accuracy_score(t_test, y))


0.0982
